# Tutorial 3. Use SEDR to do batch integration.

Here we use 3 sections from DLPFC data to show the ability of SEDR to integrate batches for the same tissue with the same techniques. 

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from tqdm import tqdm



In [2]:
import SEDR

In [3]:
data_root = Path('../../../data/DLPFC/')

proj_list = ['151673', '151674', '151675', '151676',
            '151507', '151508', '151509', '151510',
            '151669', '151670','151671', '151672']

## Combining datasets

Input of SEDR includes an AnnData object that contains the spatial transcriptomics data and a graph dictionary that contains the neighborhood graph. When combining two datasets, the AnnData objects are directly concatenated. For neighborhood graphs, we follow the following algorithm.  
Let $A^k$ and $Z_f^k$ denote the adjacency matrix and deep gene representation of spatial omics k, we then create a block-diagonal adjacency matrix $A^k$  and concatenate the deep gene representation in the spot dimension, as:  
![](batch_integration.png)  
where K is the number of spatial omics. 


In [4]:
for proj_name in tqdm(proj_list):
    adata_tmp = sc.read_visium(data_root / proj_name, count_file=proj_name+'_filtered_feature_bc_matrix.h5')
    adata_tmp.var_names_make_unique()

    adata_tmp.obs['batch_name'] = proj_name
    graph_dict_tmp = SEDR.graph_construction(adata_tmp, 12)
    
    ##### Load layer_guess label, if have
    df_label = pd.read_csv(data_root / proj_name / 'manual_annotations.txt', sep='\t', header=None, index_col=0)
    df_label.columns = ['layer_guess']
    adata_tmp.obs['layer_guess'] = df_label['layer_guess']
    
    if proj_name == proj_list[0]:
        adata = adata_tmp
        graph_dict = graph_dict_tmp
        name = proj_name
        adata.obs['proj_name'] = proj_name
    else:
        var_names = adata.var_names.intersection(adata_tmp.var_names)
        adata = adata[:, var_names]
        adata_tmp = adata_tmp[:, var_names]
        adata_tmp.obs['proj_name'] = proj_name
    
        adata = adata.concatenate(adata_tmp)
        graph_dict = SEDR.combine_graph_dict(graph_dict, graph_dict_tmp)
        name = name + '_' + proj_name

100%|██████████| 12/12 [01:25<00:00,  7.09s/it]


## Preprocessing

In [5]:
adata.layers['count'] = adata.X.toarray()
sc.pp.filter_genes(adata, min_cells=50)
sc.pp.filter_genes(adata, min_counts=10)
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", layer='count', n_top_genes=2000)
adata = adata[:, adata.var['highly_variable'] == True]
sc.pp.scale(adata)

from sklearn.decomposition import PCA  # sklearn PCA is used because PCA in scanpy is not stable. 
adata_X = PCA(n_components=200, random_state=42).fit_transform(adata.X)
adata.obsm['X_pca'] = adata_X

## Training SEDR

In [6]:
sedr_net = SEDR.Sedr(adata.obsm['X_pca'], graph_dict, mode='clustering', device='cuda:0')
using_dec = False
if using_dec:
    sedr_net.train_with_dec()
else:
    sedr_net.train_without_dec()
sedr_feat, _, _, _ = sedr_net.process()
adata.obsm['SEDR'] = sedr_feat

100%|██████████| 200/200 [00:44<00:00,  4.49it/s]


## use harmony to calculate revised PCs

In [7]:
import harmonypy as hm

meta_data = adata.obs[['batch_name']]

data_mat = adata.obsm['SEDR']
vars_use = ['batch_name']
ho = hm.run_harmony(data_mat, meta_data, vars_use)

res = pd.DataFrame(ho.Z_corr).T
res_df = pd.DataFrame(data=res.values, columns=['X{}'.format(i+1) for i in range(res.shape[1])], index=adata.obs.index)
adata.obsm[f'SEDR.Harmony'] = res_df.to_numpy()

2024-10-17 21:10:55,245 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2024-10-17 21:10:59,995 - harmonypy - INFO - sklearn.KMeans initialization complete.
2024-10-17 21:11:00,237 - harmonypy - INFO - Iteration 1 of 10
2024-10-17 21:11:14,320 - harmonypy - INFO - Iteration 2 of 10
2024-10-17 21:11:28,457 - harmonypy - INFO - Iteration 3 of 10
2024-10-17 21:11:42,537 - harmonypy - INFO - Iteration 4 of 10
2024-10-17 21:11:56,844 - harmonypy - INFO - Iteration 5 of 10
2024-10-17 21:12:08,037 - harmonypy - INFO - Iteration 6 of 10
2024-10-17 21:12:17,868 - harmonypy - INFO - Iteration 7 of 10
2024-10-17 21:12:32,121 - harmonypy - INFO - Iteration 8 of 10
2024-10-17 21:12:42,645 - harmonypy - INFO - Iteration 9 of 10
2024-10-17 21:12:52,449 - harmonypy - INFO - Iteration 10 of 10
2024-10-17 21:13:02,892 - harmonypy - INFO - Stopped before convergence


In [8]:
SEDR.mclust_R(adata, use_rep='SEDR.Harmony', key_added='mclust', n_clusters=5)

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.0.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


AnnData object with n_obs × n_vars = 47681 × 2000
    obs: 'in_tissue', 'array_row', 'array_col', 'batch_name', 'layer_guess', 'proj_name', 'batch', 'mclust'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'hvg'
    obsm: 'spatial', 'X_pca', 'SEDR', 'SEDR.Harmony'
    layers: 'count'

In [9]:
from sklearn.metrics import adjusted_rand_score

indices = np.logical_not(adata.obs["layer_guess"].isna())
ground_truth = adata.obs["layer_guess"].dropna()
ARI = adjusted_rand_score(ground_truth[indices], adata.obs['mclust'][indices])
print(f"total ARI:{ARI}")
for name in proj_list:
    sub_adata_tmp = adata[adata.obs['batch_name'] == name]
    ARI = adjusted_rand_score(sub_adata_tmp.obs['layer_guess'][indices], sub_adata_tmp.obs['mclust'][indices])
    print(f"{name} ARI:{ARI}")

total ARI:0.39741836752018983
151673 ARI:0.4765204310675322
151674 ARI:0.46054246601617305
151675 ARI:0.43705793555770134
151676 ARI:0.4648391413274263
151507 ARI:0.46858638770034
151508 ARI:0.4544795908847759
151509 ARI:0.5151843145673903
151510 ARI:0.5676860047678967
151669 ARI:0.3237846099992767
151670 ARI:0.32546618534268434
151671 ARI:0.3994067550803991
151672 ARI:0.37149854748788164


## Visualizing

### UMAP

In [ ]:
sc.pp.neighbors(adata, use_rep='SEDR.Harmony', metric='cosine')
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['layer_guess', 'batch_name'], show=False)

### LISI score

In [ ]:
ILISI = hm.compute_lisi(adata.obsm['SEDR.Harmony'], adata.obs[['batch']], label_colnames=['batch'])[:, 0]
CLISI = hm.compute_lisi(adata.obsm['SEDR.Harmony'], adata.obs[['layer_guess']], label_colnames=['layer_guess'])[:, 0]

In [ ]:
df_ILISI = pd.DataFrame({
    'method': 'SEDR',
    'value': ILISI,
    'type': ['ILISI']*len(ILISI)
})


df_CLISI = pd.DataFrame({
    'method': 'SEDR',
    'value': CLISI,
    'type': ['CLISI']*len(CLISI)
})

fig, axes = plt.subplots(1,2,figsize=(4, 5))
sns.boxplot(data=df_ILISI, x='method', y='value', ax=axes[0])
sns.boxplot(data=df_CLISI, x='method', y='value', ax=axes[1])
axes[0].set_ylim(1, 3)
axes[1].set_ylim(1, 7)
axes[0].set_title('iLISI')
axes[1].set_title('cLISI')

plt.tight_layout()